In [ ]:
!pip install python-pptx langchain langchain-community chromadb sentence_transformers \
faiss-cpu transformers accelerate tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.3 MB/s eta 0

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install python-pptx langchain langchain-community chromadb sentence_transformers \
faiss-cpu transformers accelerate tiktoken --no-deps -q

In [ ]:
!apt-get update -qq
!apt-get install -y libreoffice unoconv -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package fonts-opensymbol.
(Reading database ... 125080 files and directories currently installed.)
Preparing to unpack .../000-fonts-opensymbol_2%3a102.12+LibO7.3.7-0ubuntu0.22.04.10_all.deb ...
Unpacking fonts-opensymbol (2:102.12+LibO7.3.7-0ubuntu0.22.04.10) ...
Selecting previously unselected package libreoffice-style-colibre.
Preparing to unpack .../001-libreoffice-style-colibre_1%3a7.3.7-0ubuntu0.22.04.10_all.deb ...
Unpacking libreoffice-style-colibre (1:7.3.7-0ubuntu0.22.04.10) ...
Selecting previously unselected package libuno-sal3.
Preparing to unpack .../002-libuno-sal3_1%3a7.3.7-0ubuntu0.22.04.10_amd64.deb ...
Unpacking libuno-sal3 (1:7.3.7-0ubuntu0.22.04.10) ...
Selecting previously unsele

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDER_PATH = '/content/drive/MyDrive/ColabProjects/my_data_folder'

Mounted at /content/drive


In [ ]:
PPT_FOLDER = '/content/drive/MyDrive/content'

from pathlib import Path
import subprocess

for ppt_file in Path(PPT_FOLDER).glob("*.ppt"):
    output = ppt_file.with_suffix(".pptx")
    print(f"Converting: {ppt_file.name} → {output.name}")
    subprocess.run(["soffice", "--headless", "--convert-to", "pptx", str(ppt_file), "--outdir", PPT_FOLDER])

Converting: Lecture 4-ranking.ppt → Lecture 4-ranking.pptx
Converting: lecture week 4-evaluation.ppt → lecture week 4-evaluation.pptx
Converting: Data Mining-classification review.ppt → Data Mining-classification review.pptx
Converting: lecture-textcat-nbayes.ppt → lecture-textcat-nbayes.pptx
Converting: lecture-SVMs.ppt → lecture-SVMs.pptx
Converting: lecture-clustering 1.ppt → lecture-clustering 1.pptx


In [ ]:
def clean_text(text):
    text = text.replace("\n\n", "\n")
    text = text.replace("•", "-")
    text = text.replace("\t", " ")
    text = "\n".join([line.strip() for line in text.splitlines() if line.strip() != ""])
    return text

In [ ]:
from pptx import Presentation
from pathlib import Path
from langchain.docstore.document import Document

PPT_FOLDER = '/content/drive/MyDrive/content'
documents = []

for ppt_file in Path(PPT_FOLDER).glob("*.pptx"):
    prs = Presentation(str(ppt_file))
    for i, slide in enumerate(prs.slides):
        text = []
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                text.append(clean_text(shape.text))

        if text:
            documents.append(
                Document(
                    page_content="\n".join(text),
                    metadata={"source_file": ppt_file.name, "slide": i+1}
                )
            )

print(" Slides loaded:", len(documents))

 Slides loaded: 689


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
chunks = splitter.split_documents(documents)

print(" Total text chunks:", len(chunks))


 Total text chunks: 733


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

PERSIST_DIR = "/content/chroma_ppt_store"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(chunks, embeddings, persist_directory=PERSIST_DIR)
vectordb.persist()

retriever = vectordb.as_retriever(search_kwargs={"k": 4})
print("Vector database created and retriever ready.")

/tmp/ipython-input-2623924369.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector database created and retriever ready.


/tmp/ipython-input-2623924369.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
import os
from langchain.chat_models import ChatOpenAI

import os

# Replace YOUR_KEY_HERE with your actual key inside quotes
os.environ["OPENAI_API_KEY"] = # Add your OpenAPI key

#from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",  # or gpt-4o / gpt-3.5-turbo
    temperature=0.3,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

#llm = ChatOpenAI(
#    model_name="gpt-4o-mini", # high quality + low cost
#    temperature=0             # temperature=0 → accurate & stable
#)

print("LLM ready.")


/tmp/ipython-input-3321602370.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


LLM ready.


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
You are a helpful tutor. Answer the question using ONLY the slide content provided.
If the answer is not in the slides, say: "I don't know based on the slides."

Question: {question}

Slides:
{context}

Answer:
""")

def ask(question):
    docs = retriever.get_relevant_documents(question)

    context = "\n\n---\n\n".join(
        [f"[Slide {d.metadata['slide']} — {d.metadata['source_file']}]\n{d.page_content}"
         for d in docs]
    )

    final_prompt = prompt.format(question=question, context=context)
    response = llm.invoke(final_prompt)  # invoke() is correct for ChatOpenAI

    return response.content, docs

In [ ]:
while True:
    q = input("\nAsk a question (or type 'exit' to stop): ")
    if q.lower() in ["exit", "quit", "stop"]:
        break

    answer, sources = ask(q)

    print("\n Answer:\n", answer)
    print("\n Sources:")
    for d in sources:
        print(f"- {d.metadata['source_file']} (Slide {d.metadata['slide']})")



Ask a question (or type 'exit' to stop): What is Information Retrieval


/tmp/ipython-input-1600675183.py:16: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)



 Answer:
 Information Retrieval (IR) is finding material (usually documents) of an unstructured nature (usually text) that satisfies an information need from within large collections (usually stored on computers).

 Sources:
- lecture week1 -in-person.pptx (Slide 4)
- recommendation sys.pptx (Slide 62)
- lecture week1 -in-person.pptx (Slide 7)
- lecture-textcat-nbayes.pptx (Slide 1)

Ask a question (or type 'exit' to stop): What is cosine similarity

 Answer:
 Cosine similarity is the cosine of the angle between two vectors, specifically calculated as the dot product of the vectors. It is represented as cos(q,d), where q is the query and d is the document.

 Sources:
- cosine similarity example.pptx (Slide 3)
- term weighting and scoring new.pptx (Slide 32)
- Lecture 4-ranking.pptx (Slide 13)
- Document-Summarization-Intro.pptx (Slide 18)

Ask a question (or type 'exit' to stop): exit
